In [1]:
from catboost import Pool, cv, CatBoostClassifier
from constants import *
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import optuna

# Load Data

In [2]:
features_df = pd.read_csv(TRAINING_FEATURES_PATH, index_col='respondent_id')
labels_df = pd.read_csv(TRAINING_LABELS_PATH,index_col='respondent_id')
test_features_df = pd.read_csv(TEST_FEATURES_PATH, index_col="respondent_id")

# Impute

In [3]:
num_cols = features_df.select_dtypes('number').columns
cat_cols = ['race', 'sex', 'marital_status', 'rent_or_own',  'hhs_geo_region', 
            'census_msa', 'employment_industry', 'employment_occupation']
ord_cols = ['age_group', 'education', 'income_poverty', 'employment_status']

In [4]:
for col in (cat_cols+ord_cols):
    features_df[col] = features_df[col].fillna(value='None')
    test_features_df[col] = test_features_df[col].fillna(value='None')

for col in num_cols:
    features_df[col] = features_df[col].fillna(value=-1)
    test_features_df[col] = test_features_df[col].fillna(value=-1)
    

In [5]:
categorical_features_indices = np.where(features_df.dtypes != float)[0]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(features_df, labels_df, test_size=0.3, random_state=68)

In [7]:
def objective(trial: optuna.Trial, pool: Pool) -> float:
    params = {
        'iterations': trial.suggest_categorical('iterations', [100, 200, 300, 500, 1000, 1200, 1500]),
        'learning_rate': trial.suggest_float("learning_rate", 0.001, 0.3),
        'random_strength': trial.suggest_int("random_strength", 1, 10),
        'bagging_temperature': trial.suggest_int("bagging_temperature", 0, 10),
        'max_bin': trial.suggest_categorical('max_bin', [4, 5, 6, 8, 10, 20, 30]),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10),
        "depth": trial.suggest_int("max_depth", 2,10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-8, 100),
        'one_hot_max_size': trial.suggest_categorical('one_hot_max_size', [5,10,12,100,500,1024]),
        'auto_class_weights': trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        'custom_metric': ['AUC'],
        'od_type': "Iter",
        'od_wait': 100,
        "loss_function": "Logloss",
    }

    scores = cv(
        pool=pool,
        params=params,
        fold_count=5, 
        early_stopping_rounds=10,         
        plot=False, 
        logging_level='Silent'
    )

    return scores['test-AUC-mean'].max()

# H1N1 Vaccine Prediction

In [8]:
h1n1_vaccine_pool = Pool(data=X_train,
                     label=y_train.h1n1_vaccine,
                     cat_features = categorical_features_indices)

In [9]:
sampler = optuna.samplers.TPESampler(seed=68)  # Make the sampler behave in a deterministic way.
study_h1n1 = optuna.create_study(direction="maximize", sampler=sampler)
study_h1n1.optimize(lambda trial: objective(trial, h1n1_vaccine_pool), n_trials=100)

[I 2023-04-27 17:10:45,272] A new study created in memory with name: no-name-b0fdea65-ceda-4f1f-ba77-d23a319cfa0d
[I 2023-04-27 17:11:14,296] Trial 0 finished with value: 0.8655198294622981 and parameters: {'iterations': 1500, 'learning_rate': 0.029356482739949695, 'random_strength': 8, 'bagging_temperature': 10, 'max_bin': 6, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 1, 'max_depth': 4, 'l2_leaf_reg': 52.99113765425227, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.8655198294622981.
[I 2023-04-27 17:11:25,802] Trial 1 finished with value: 0.8653117586190222 and parameters: {'iterations': 200, 'learning_rate': 0.1464067066361795, 'random_strength': 10, 'bagging_temperature': 3, 'max_bin': 10, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 1, 'max_depth': 3, 'l2_leaf_reg': 64.53360776051433, 'one_hot_max_size': 500, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.8655198294622981.
[I 2023-04-27 17:11:46,258] 

In [10]:
print("Number of finished trials: {}".format(len(study_h1n1.trials)))
print("Best trial:")
trial = study_h1n1.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}={},".format(key, value))

Number of finished trials: 100
Best trial:
  Value: 0.868372165211389
  Params: 
    iterations=1500,
    learning_rate=0.015494337354824417,
    random_strength=7,
    bagging_temperature=6,
    max_bin=5,
    grow_policy=Depthwise,
    min_data_in_leaf=10,
    max_depth=10,
    l2_leaf_reg=80.31439723807587,
    one_hot_max_size=5,
    auto_class_weights=SqrtBalanced,


In [11]:
final_model_h1n1 = CatBoostClassifier(
    verbose=False,  
    cat_features=categorical_features_indices, 
    **trial.params
)

final_model_h1n1.fit(X_train, y_train.h1n1_vaccine)
predictions_h1n1 = final_model_h1n1.predict_proba(X_test)
predictions_h1n1 = predictions_h1n1[:,1].reshape(-1,1)
roc_auc_score(y_test.h1n1_vaccine, predictions_h1n1)

0.8745725797935606

# Seasonal Vaccine Prediction

In [8]:
seasonal_vaccine_pool = Pool(
    data=X_train,
    label=y_train.seasonal_vaccine,
    cat_features = categorical_features_indices
)

In [ ]:
sampler = optuna.samplers.TPESampler(seed=68)  # Make the sampler behave in a deterministic way.
study_seasonal = optuna.create_study(
    study_name='seasonal-study', 
    storage='sqlite:///seasonal.db', 
    load_if_exists=True,
    direction="maximize", 
    sampler=sampler
)
study_seasonal.optimize(lambda trial: objective(trial, seasonal_vaccine_pool), n_trials=100)

In [10]:
study_seasonal = optuna.create_study(
    study_name='seasonal-study', 
    storage='sqlite:///seasonal.db', 
    load_if_exists=True,
)

print("Number of finished trials: {}".format(len(study_seasonal.trials)))
print("Best trial:")
trial2 = study_seasonal.best_trial
print("  Value: {}".format(trial2.value))
print("  Params: ")
for key, value in trial2.params.items():
    print("    {}={},".format(key, value))

[I 2023-04-28 17:27:46,282] Using an existing study with name 'seasonal-study' instead of creating a new one.


Number of finished trials: 103
Best trial:
  Value: 0.8637805392567849
  Params: 
    auto_class_weights=SqrtBalanced,
    bagging_temperature=0,
    grow_policy=Depthwise,
    iterations=1500,
    l2_leaf_reg=77.31367796732783,
    learning_rate=0.03257791879887687,
    max_bin=20,
    max_depth=8,
    min_data_in_leaf=1,
    one_hot_max_size=500,
    random_strength=2,


In [22]:
final_model_seasonal = CatBoostClassifier(
    verbose=False,  
    cat_features=categorical_features_indices, 
    **study_seasonal.best_trial.params,
    od_type="Iter",
    od_wait=10,
    loss_function="Logloss",
    custom_metric=['AUC'],
)

final_model_seasonal.fit(X_train, y_train.seasonal_vaccine)
predictions_seasonal = final_model_seasonal.predict_proba(X_test)
predictions_seasonal = predictions_seasonal[:,1].reshape(-1,1)
roc_auc_score(y_test.seasonal_vaccine, predictions_seasonal)

0.8588832298640332

In [16]:
scores = cv(
    pool=seasonal_vaccine_pool,
    params=study_seasonal.best_trial.params | {
        'custom_metric': ['AUC'],
        'od_type': "Iter",
        'od_wait': 100,
        "loss_function": "Logloss",
    },
    fold_count=5, 
    early_stopping_rounds=10,         
    plot=False, 
    logging_level='Silent'
)

In [19]:
scores['test-AUC-mean'].max()

0.8637805392567849

In [76]:
# Let's see the score combined of the two best predictions
roc_auc_score(y_test, np.hstack((predictions_h1n1, predictions_seasonal)))

0.8685882060060544

# Retrain on full data

In [17]:
final_model_h1n1.fit(features_df, labels_df.h1n1_vaccine)
final_h1n1 = final_model_h1n1.predict_proba(test_features_df)
final_h1n1 = final_h1n1[:,1].reshape(-1,1)

In [18]:
final_model_seasonal.fit(features_df, labels_df.seasonal_vaccine)
final_se = final_model_seasonal.predict_proba(test_features_df)
final_se = final_se[:,1].reshape(-1,1)

In [19]:
submission_df = pd.read_csv(
    SUBMISSION_FORMAT_PATH, 
    index_col="respondent_id"
)
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = final_h1n1
submission_df["seasonal_vaccine"] = final_se

submission_df.to_csv(SUBMISSION_DIR / 'catboost-optuna-2.csv', index=True)

# Bagging Model with K-Fold CrossValidation

In [25]:
### FOR H1N1

from sklearn.model_selection import KFold

K = 5
kf = KFold(n_splits=K)

models: list[tuple[CatBoostClassifier, float]] = []

for i, (train_index, test_index) in enumerate(kf.split(features_df)):
    X_train = features_df.iloc[train_index]
    X_test  = features_df.iloc[test_index]
    y_train = labels_df.iloc[train_index]
    y_test  = labels_df.iloc[test_index]

    model = CatBoostClassifier(cat_features=categorical_features_indices, verbose=False, **trial.params)

    model.fit(X_train, y_train.h1n1_vaccine)
    probabilities = model.predict_proba(X_test)[:,1].reshape(-1,1)
    weight = roc_auc_score(y_test.h1n1_vaccine, probabilities)
    models.append((model, weight))
    print(f"[FOLD {i+1}/{K}] DONE.")


[FOLD 0/5] DONE.
[FOLD 1/5] DONE.
[FOLD 2/5] DONE.
[FOLD 3/5] DONE.
[FOLD 4/5] DONE.


In [28]:
def bag_predict(models: list[tuple[CatBoostClassifier, float]], X):
    norm = sum(weight for _, weight in models)

    preds = np.zeros((X.shape[0], 1))
    
    for model, weight in models:
        preds += (weight/norm) * model.predict_proba(X)[:,1].reshape(-1,1)

    return preds


roc_auc_score(y_test.h1n1_vaccine, bag_predict(models, X_test))

0.9085078185892029

In [52]:
### For seasonal Vaccine

from sklearn.model_selection import KFold

K = 5
kf = KFold(n_splits=K)

models_seasonal: list[tuple[CatBoostClassifier, float]] = []

for i, (train_index, test_index) in enumerate(kf.split(features_df)):
    X_train = features_df.iloc[train_index]
    X_test  = features_df.iloc[test_index]
    y_train = labels_df.iloc[train_index]
    y_test  = labels_df.iloc[test_index]

    model = CatBoostClassifier(cat_features=categorical_features_indices, verbose=False, **trial2.params)

    model.fit(X_train, y_train.seasonal_vaccine)
    probabilities = model.predict_proba(X_test)[:,1].reshape(-1,1)
    weight = roc_auc_score(y_test.seasonal_vaccine, probabilities)
    models_seasonal.append((model, weight))
    print(f"[FOLD {i+1}/{K}] DONE.")


[FOLD 1/5] DONE.
[FOLD 2/5] DONE.
[FOLD 3/5] DONE.
[FOLD 4/5] DONE.
[FOLD 5/5] DONE.


In [54]:
roc_auc_score(y_test, np.hstack((bag_predict(models, X_test), bag_predict(models_seasonal, X_test))))

0.8956539156114287

In [56]:
submission_df = pd.read_csv(
    SUBMISSION_FORMAT_PATH, 
    index_col="respondent_id"
)
np.testing.assert_array_equal(test_features_df.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = bag_predict(models, test_features_df)
submission_df["seasonal_vaccine"] = bag_predict(models_seasonal, test_features_df)

submission_df.to_csv(SUBMISSION_DIR / 'catboost-optuna-bagging-1.csv', index=True)

In [63]:
import pickle

H1N1_BAG_PATH = Path('..') / 'models' / 'catboost' / 'h1n1' / f'catboost-bagged-5-optuna-100'
SEASONAL_BAG_PATH = Path('..') / 'models' / 'catboost' / 'seasonal' / f'catboost-bagged-5-optuna-10'

for i in range(K):
    with open(H1N1_BAG_PATH, 'wb') as f:
        pickle.dump(models, f)
        
    with open(SEASONAL_BAG_PATH, 'wb') as f:
        pickle.dump(models_seasonal, f)

In [68]:
import pickle

with open(H1N1_BAG_PATH, 'rb') as f:
    h1n1_bag = pickle.load(f)

with open(SEASONAL_BAG_PATH, 'rb') as f:
    seasonal_bag = pickle.load(f)

roc_auc_score(y_test, np.hstack((bag_predict(h1n1_bag, X_test), bag_predict(seasonal_bag, X_test))))

0.8956539156114287